### Importing libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

### Reading Dataset

In [23]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [24]:
# Preprocessing the data
# Dropping irrelevant columns

data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [25]:
# Converting categorical variables into numbers

# Converting gender into category
label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])

In [26]:
# Converting Geography into category

from sklearn.preprocessing import OneHotEncoder

onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [27]:
column_names = onehot_encoder_geo.get_feature_names_out(["Geography"])

In [28]:
geo_encoder_df = pd.DataFrame(geo_encoder,columns=column_names)

In [29]:
data = pd.concat([data.drop(["Geography"],axis=1),geo_encoder_df],axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [61]:
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo,file) 

In [33]:
# Divide the dataset into dependent and independent variables
X = data.drop(["Exited"],axis=1)
y = data["Exited"]

# Split data in training and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [36]:
with open("scalar.pkl","wb") as file:
    pickle.dump(scaler,file)

### ANN Impletation

In [38]:
import tensorflow
from tensorflow.keras.models import Sequential;
from tensorflow.keras.layers import Dense;
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [39]:
X_train.shape

(8000, 12)

In [40]:
# Building ANN model

model = Sequential(
    [
        Dense(64,activation="relu",input_shape=(X_train.shape[1],)), #HL1 Connected with input layer
        Dense(32,activation="relu"), #HL2 Conected
        Dense(1,activation="sigmoid") #Output layer
    ]
)

/opt/miniconda3/envs/ANNProject/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
# For optimizer and loss, if we want to change the optimizer learning rate
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [ ]:
# Compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [53]:
# Setup tensorboard
log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [54]:
#Setup Early Stopping
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [55]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
                    callbacks=[tensorflow_callback,early_stopping_callback]
                    )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step - accuracy: 0.8695 - loss: 0.3198 - val_accuracy: 0.8605 - val_loss: 0.3382
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - accuracy: 0.8646 - loss: 0.3318 - val_accuracy: 0.8600 - val_loss: 0.3572
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 477us/step - accuracy: 0.8699 - loss: 0.3216 - val_accuracy: 0.8560 - val_loss: 0.3560
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 476us/step - accuracy: 0.8669 - loss: 0.3220 - val_accuracy: 0.8565 - val_loss: 0.3512
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 478us/step - accuracy: 0.8666 - loss: 0.3155 - val_accuracy: 0.8575 - val_loss: 0.3579
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step - accuracy: 0.8688 - loss: 0.3171 - val_accuracy: 0.8535 - val_loss: 0.3565
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 477us/step - accuracy: 0.8736 - loss: 0.3070 - val_accuracy: 0.8590 - val_loss: 0.3476
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - accuracy: 0.8679 - loss: 0

In [56]:
model.save("model.h5")

In [57]:
%load_ext tensorboard

In [60]:
%tensorboard --logdir log/fit/